# Pryngles module

In [7]:
from pryngles import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## External modules

## Planet class

In [8]:
class PlanetDefaults(object):
    """
    These are the default attributes for bodies of the kind 'Planet'.
    
    DEVELOPER:
        You may add new attributes as the model gets more complex.
        Please document properly each attribute.
        
    orbit:
    
        These attributes should be compatible with rebound.
    
        m: float [rebound mass units], default = 1
            Mass of the body.  If m = 0 the body does not produce gravitation.
            
        a: float [rebound length units], default = 1.0
            Semi major axis of the orbit with respect to primary.
            
        e: float [adimentional], default = 0.0
            Eccentricity of the orbit with respect to primary.
            
    physics:
    
        radius: float [rebound length units], default = 1
            Radius of the body.
            
        prot: float [rebound time units], default = 1
            Period of rotation
            
    optics:
    
        nspangles: int, default = 1000
            Number of spangles on which the object will be discretized.
    """

    orbit=dict(m=1.0,a=1.0,e=0.0)
    
    physics=dict(radius=1.0,prot=1.0)
    
    optics=dict(nspangles=1000)

In [9]:
BODY_KINDS+=["Planet"]

class Planet(Body):
    """Class Planet.
    
    See Body class documentation.
    
    Additional public attributes:
    
        physics.wrot: float
            Rotational angular velocity
    
    Override methods:
    
        update_body(**pars):
            This method compute additional attributes like (see above).
    """
    def __init__(self,
                 primary=None,
                 orbit=PlanetDefaults.orbit,
                 physics=PlanetDefaults.physics,
                 optics=PlanetDefaults.optics
                ):
        
        
        #Instantiate object with basic properties
        Body.__init__(self,PlanetDefaults,"Planet",primary,orbit,physics,optics)
        
        #Check primary
        if self.primary is None:
            raise ValueError(f"Primary not provided and it is mandatory for {self.kind}.")
        self.primary=primary
        self.primary._update_childs(self)
        self._update_parent(self.primary)
        
        #Update properties
        self.update_body(**self.__dict__)
        
    def update_body(self,**pars):
        Body.update_body(self,**pars)
        
        #Here place the commands to update this kind of body
        self.physics.wrot=2*np.pi/self.physics.prot

In [10]:
if IN_JUPYTER:
    def test_planet(self):
        S=Star()

        #Check exception: primary is mandatory for planets
        self.assertRaises(ValueError,lambda:Planet())

        P=Planet(primary=S)
        
        print(P.physics)
        print(P.hash)
        
        #Check derived properties
        self.assertEqual(np.isclose([P.physics.wrot],
                                    [2*np.pi/PlanetDefaults.physics["prot"]],
                                    rtol=1e-7),
                         [True]*1)
        
        P.update_body(orbit=dict(a=5),physics=dict(rho=0.2))
        print(P.orbit,P.physics)
        
        #Check exception: primary could not be different from None or Body
        self.assertRaises(AssertionError,lambda:Planet(primary="Nada"))
        
    class Test(unittest.TestCase):pass    
    Test.test_planet=test_planet
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

{'radius': 1.0, 'prot': 1.0, 'wrot': 6.283185307179586}
8776926439708
{'m': 1.0, 'a': 5, 'e': 0.0} {'radius': 1.0, 'prot': 1.0, 'wrot': 6.283185307179586, 'rho': 0.2}



----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


--End--